In [8]:
import seaborn as sns
import pandas as pd
import folium
import numpy as np 
import scipy
import matplotlib
import matplotlib.pyplot as plt
from prettytable import PrettyTable
import re
import os
import warnings
import plotly.express as px
import geopandas as gpd
from scipy import stats
from datetime import datetime

warnings.filterwarnings('ignore')
%matplotlib inline

In [9]:
print("NumPy version:", np.__version__)
print("Pandas version:", pd.__version__)
print("Matplotlib version:", matplotlib.__version__)
print("Seaborn version:", sns.__version__)
print("scipy version:", scipy.__version__)

NumPy version: 2.0.1
Pandas version: 2.2.2
Matplotlib version: 3.9.1.post1
Seaborn version: 0.13.2
scipy version: 1.14.0


In [11]:
# @deprecated
def ensure_dir_exists(directory):
    """
    Create a directory if it doesn't exist
    """
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Created directory: {directory}")

# @deprecated
def save_to_question_dir(filename, question_num, *args, **kwargs):
    """
    Save a file to the specified question directory, creating it if necessary
    """
    dir_name = f'q{question_num}'
    ensure_dir_exists(dir_name)
    filepath = os.path.join(dir_name, filename)
    plt.savefig(filepath, *args, **kwargs)
    print(f"Saved {filename} to {filepath}")

### Reading Property data

In [12]:
data = pd.read_csv('properties.csv', low_memory=False)

### Columns and dtypes (for reference)

In [13]:
for col in data.columns:
    print(f"Column '{col}': {data[col].dtype}")

Column 'ID': int64
Column 'Possession Status': object
Column 'Availability Starts From': object
Column 'Floor No': object
Column 'Commercial': object
Column 'Developer': object
Column 'Approved Authority Name': object
Column 'Units Available': float64
Column 'Price': float64
Column 'Price (English)': object
Column 'Flooring Type': object
Column 'Electricity Status': object
Column 'Maintenance Type': object
Column 'Maintenance Charges': float64
Column 'Booking Amount': float64
Column 'Landmark': object
Column 'Covered Area': float64
Column 'Project Name': object
Column 'sqft Price ': float64
Column 'Carpet Area': float64
Column 'Area Name': object
Column 'Property Uniqueness': object
Column 'Unit of Carpet Area': object
Column 'Society': object
Column 'Ownership Type': object
Column 'furnished Type': object
Column 'Bathroom': float64
Column 'Parking': object
Column 'Facing': object
Column 'Amenities Facing': object
Column 'Transaction Type': object
Column 'Floor Data': object
Column 'Ty

## Question 1

In [ ]:
def determine_budget_category(price, price_thresholds):
    """
    Categorize a property's price into budget categories.
    
    Args:
    price (float): The price of the property.
    price_thresholds (pandas.Series): Series containing the 25th and 75th percentile prices.
    
    Returns:
    str: The budget category ('Low', 'Medium', 'High', or 'Unknown').
    """
    if pd.isnull(price):
        return "Unknown"
    elif price <= price_thresholds.iloc[0]:
        return "Low"
    elif price <= price_thresholds.iloc[1]:
        return "Medium"
    else:
        return "High"

def create_budget_distribution_chart(city_data, city_name):
    """
    Create a bar chart showing the distribution of budget categories for a city.
    
    Args:
    city_data (pandas.DataFrame): Data for the specific city.
    city_name (str): Name of the city (or 'All Cities' for combined data).
    """
    all_budget_categories = ["Low", "Medium", "High", "Unknown"]
    category_counts = city_data["Budget_Category"].value_counts()
    category_counts = category_counts.reindex(all_budget_categories, fill_value=0)

    plt.figure(figsize=(10, 7))
    bars = plt.bar(
        category_counts.index,
        category_counts.values,
        color=["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728"]
    )

    plt.xlabel("Budget Category")
    plt.ylabel("Number of Properties")
    plt.title(f"Distribution of Budget Categories in {city_name}")

    for bar in bars:
        bar_height = bar.get_height()
        plt.text(
            bar.get_x() + bar.get_width() / 2,
            bar_height,
            int(bar_height),
            ha="center", va="bottom",
            color="black", fontsize=12
        )

    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()
    plt.close()

def analyze_and_visualize_budget_categories(property_data):
    """
    Analyze property prices, categorize them into budget ranges, and create visualizations.
    
    Args:
    property_data (pandas.DataFrame): The dataset containing property information.
    """
    price_percentiles = [25, 75]
    price_thresholds = property_data["Price"].quantile([p / 100 for p in price_percentiles])
    property_data["Budget_Category"] = property_data["Price"].apply(
        lambda price: determine_budget_category(price, price_thresholds)
    )

    overall_distribution = property_data["Budget_Category"].value_counts()
    print("Overall Budget Distribution:")
    print(overall_distribution)

    for city in property_data["City"].unique():
        city_specific_data = property_data[property_data["City"] == city]
        create_budget_distribution_chart(city_specific_data, city)
    
    create_budget_distribution_chart(property_data, "All Cities")
    city_budget_breakdown = property_data.groupby(["City", "Budget_Category"]).size().unstack(fill_value=0)
    city_budget_breakdown = city_budget_breakdown.reindex(columns=["Low", "Medium", "High", "Unknown"])

    plt.figure(figsize=(15, 10))
    city_budget_breakdown.plot(kind="bar", stacked=False)
    plt.xlabel("City")
    plt.ylabel("Number of Properties")
    plt.title("Distribution of Budget Categories by City")
    plt.legend(title="Budget Category", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.xticks(rotation=45, ha="right")
    plt.tight_layout()
    plt.show()
    plt.close()

analyze_and_visualize_budget_categories(data)

### `Inferences & Explanations`

* Initially divided all the prices into categories based on percentiles (25 & 75) - *low, medium, high, unknown*

* Plots for each city showing number of properties v/s price categories

* Final plot shows comparative analysis of **number of properties for each city for each budget** range in a single graph using grouped bra chart


-> We can notice from the final graph, if an investor wants to particularly invest in a low budget position then he/she may find numerous suitable opporutinities in _Mumbai & Thane_

-> Similarly, for high end budget investors, by a large margin, _Mumbai_ offers the most opportunities. The same also goes for medium range also

-> In addition, **Thane** is also providing decent number of properties for each of the 3 categories of investor budgets

-> Although, the distributions overall across cities is highly skewed towards both of these cities as all other cities like `Agartala, Gurgaon, Kalyan etc.` have mere 100s of properties of each type while those in Mumbai range to almost _4k_ properties

## Question 2

In [ ]:
def categorize_property_price(price, price_thresholds):
    """
    Categorize a property's price into budget categories.
    
    Args:
    price (float): The price of the property.
    price_thresholds (pandas.Series): Series containing the 25th and 75th percentile prices.
    
    Returns:
    str: The budget category ('Low', 'Medium', 'High', or 'Unknown').
    """
    if pd.isnull(price):
        return "Unknown"
    elif price <= price_thresholds.iloc[0]:
        return "Low"
    elif price <= price_thresholds.iloc[1]:
        return "Medium"
    else:
        return "High"

def analyze_and_visualize_property_data(property_data):
    """
    Perform comprehensive analysis and visualization on property data.
    
    Args:
    property_data (pandas.DataFrame): The dataset containing property information.
    """
    df = property_data.copy()
    price_percentiles = [25, 75]
    price_thresholds = df["Price"].quantile([p / 100 for p in price_percentiles])
    df["Price_Category"] = df["Price"].apply(
        lambda price: categorize_property_price(price, price_thresholds)
    )
    
    create_property_distribution_heatmap(df)
    create_amenities_distribution_chart(df)
    create_price_vs_size_scatter_plot(df)
    display_summary_statistics(df)

def create_property_distribution_heatmap(df):
    """Create a heatmap showing property distribution across cities, price categories, and property types."""
    plt.figure(figsize=(14, 12))
    heatmap_data = pd.crosstab(df["City"], [df["Price_Category"], df["Type of Property"]])
    sns.heatmap(heatmap_data, annot=True, fmt="d", cmap="YlOrRd")
    plt.title("Property Distribution: Price Range vs City vs Property Type", fontsize=16)
    plt.tight_layout()
    plt.show()
    plt.close()

def create_amenities_distribution_chart(df):
    """Create a bar chart showing the distribution of key amenities across price categories."""
    key_amenities = ["Swimming Pool", "Gymnasium", "Club House", "Security", "Lift"]
    amenity_data = df.groupby("Price_Category")[key_amenities].mean()
    
    plt.figure(figsize=(14, 8))
    amenity_data.plot(kind="bar", width=0.8)
    plt.title("Key Amenities Distribution by Price Range", fontsize=16)
    plt.xlabel("Price Range", fontsize=12)
    plt.ylabel("Proportion of Properties with Amenity", fontsize=12)
    plt.legend(title="Amenities", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()
    plt.show()
    plt.close()

def create_price_vs_size_scatter_plot(df):
    """Create a scatter plot of property price vs size, differentiated by city and number of bedrooms."""
    # Appling log transformation to Price and Covered Area for better visualization (Fixing more outliers)
    df["Log_Price"] = np.log10(df["Price"])
    df["Log_Area"] = np.log10(df["Covered Area"])
    
    plt.figure(figsize=(14, 10))
    for city in df["City"].unique():
        city_data = df[df["City"] == city]
        plt.scatter(
            city_data["Log_Area"],
            city_data["Log_Price"],
            alpha=0.6,
            label=city,
            s=city_data["bedroom"] * 10,
        )
    plt.title("Property Price vs Size by City (Log-Scaled)", fontsize=16)
    plt.xlabel("Log10(Covered Area) (sq ft)", fontsize=12)
    plt.ylabel("Log10(Price) (INR)", fontsize=12)
    plt.legend(title="City", fontsize=10)
    plt.tight_layout()
    plt.show()
    plt.close()

def display_summary_statistics(df):
    """Generate and display summary statistics for properties in each price category."""
    summary_stats = (
        df.groupby("Price_Category")
        .agg(
            {
                "Price": ["count", "mean", "median"],
                "Covered Area": "mean",
                "bedroom": "mean",
            }
        )
        .round(2)
    )
    
    table = PrettyTable()
    table.field_names = ["Price Category"] + [
        f"{col[0]}_{col[1]}".strip() if col[1] else col[0] for col in summary_stats.columns
    ]
    
    for index, row in summary_stats.iterrows():
        table.add_row([index] + row.tolist())
    
    print("\nSummary Statistics by Price Category:")
    print(table)

analyze_and_visualize_property_data(data)

## Question 3

In [ ]:
def convert_area_to_sqft(area, unit):
    """
    Convert the given area to square feet based on the unit.

    Args:
    area (float): The area value to be converted.
    unit (str): The unit of the area ('Sq-ft', 'Sq-m', or 'Sq-yrd').

    Returns:
    float or None: The area in square feet, or None if input is invalid.
    """
    if pd.isna(area) or pd.isna(unit):
        return None
    conversion_factors = {'Sq-ft': 1, 'Sq-m': 10.7639, 'Sq-yrd': 9}
    return area * conversion_factors.get(unit, 0)

def remove_outliers_iqr(df, columns):
    """
    Remove outliers from specified columns using the Interquartile Range (IQR) method.

    Args:
    df (pandas.DataFrame): The dataframe to process.
    columns (list): List of column names to remove outliers from.

    Returns:
    pandas.DataFrame: The dataframe with outliers removed.
    """
    for column in columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        df = df[(df[column] >= Q1 - 1.5 * IQR) & (df[column] <= Q3 + 1.5 * IQR)]
    return df

def analyze_thane_mumbai_property_data(data):
    """
    Analyze and visualize property data for Thane and Mumbai.

    Args:
    data (pandas.DataFrame): The raw property data.
    """
    carpet_area_df = data[['City', 'Unit of Carpet Area', 'Carpet Area', 'Carpet Area Unit', 'Type of Property', 'Price']].copy()
    
    print("Null values in each column:")
    print(carpet_area_df.isnull().sum())
    
    carpet_area_df_clean = carpet_area_df.dropna()
    carpet_area_df_clean = carpet_area_df_clean[carpet_area_df_clean['Unit of Carpet Area'] == carpet_area_df_clean['Carpet Area Unit']]
    
    carpet_area_df_clean['Carpet Area (Sq-ft)'] = carpet_area_df_clean.apply(
        lambda row: convert_area_to_sqft(row['Carpet Area'], row['Carpet Area Unit']), axis=1
    )
    carpet_area_df_clean = carpet_area_df_clean.dropna(subset=['Carpet Area (Sq-ft)'])
    
    cities_data = carpet_area_df_clean[carpet_area_df_clean['City'].isin(['Thane', 'Mumbai'])].copy()
    cities_data['Price per sq ft'] = cities_data['Price'] / cities_data['Carpet Area (Sq-ft)']
    
    mumbai_data = remove_outliers_iqr(cities_data[cities_data['City'] == 'Mumbai'], ['Carpet Area (Sq-ft)', 'Price'])
    thane_data = remove_outliers_iqr(cities_data[cities_data['City'] == 'Thane'], ['Carpet Area (Sq-ft)', 'Price'])
    cleaned_cities_data = pd.concat([mumbai_data, thane_data])
    
    create_property_type_distribution_chart(cleaned_cities_data)
    create_carpet_area_distribution_chart(cleaned_cities_data)
    create_price_distribution_chart(cleaned_cities_data)
    create_carpet_area_vs_price_charts(cleaned_cities_data)
    create_avg_price_by_property_type_chart(cleaned_cities_data)

def create_property_type_distribution_chart(data):
    """Create a stacked bar chart showing property type distribution in Thane and Mumbai."""
    property_type_dist = data.groupby(['City', 'Type of Property']).size().unstack(fill_value=0)
    property_type_dist_pct = property_type_dist.div(property_type_dist.sum(axis=1), axis=0)
    
    plt.figure(figsize=(12, 6))
    property_type_dist_pct.plot(kind='bar', stacked=True)
    plt.title('Property Type Distribution in Thane and Mumbai', fontsize=16)
    plt.xlabel('City', fontsize=12)
    plt.ylabel('Percentage', fontsize=12)
    plt.legend(title='Property Type', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()
    plt.close()

def create_carpet_area_distribution_chart(data):
    """Create a box plot showing carpet area distribution in Thane and Mumbai."""
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='City', y='Carpet Area (Sq-ft)', data=data)
    plt.title('Carpet Area Distribution in Thane and Mumbai', fontsize=16)
    plt.xlabel('City', fontsize=12)
    plt.ylabel('Carpet Area (Sq-ft)', fontsize=12)
    plt.show()
    plt.close()

def create_price_distribution_chart(data):
    """Create a box plot showing price per sq ft distribution in Thane and Mumbai."""
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='City', y='Price per sq ft', data=data)
    plt.title('Price per sq ft Distribution in Thane and Mumbai', fontsize=16)
    plt.xlabel('City', fontsize=12)
    plt.ylabel('Price per sq ft', fontsize=12)
    plt.show()
    plt.close()

def create_carpet_area_vs_price_charts(data):
    """Create scatter plots of carpet area vs price for Thane and Mumbai, in both linear and log scales."""
    fig, axes = plt.subplots(2, 2, figsize=(20, 16))
    
    for i, city in enumerate(['Thane', 'Mumbai']):
        city_data = data[data['City'] == city]
        
        # Linear scale plot
        sns.scatterplot(x='Carpet Area (Sq-ft)', y='Price', data=city_data, alpha=0.6, ax=axes[0, i])
        axes[0, i].set_title(f'{city}: Carpet Area vs Price', fontsize=14)
        axes[0, i].set_xlabel('Carpet Area (Sq-ft)', fontsize=12)
        axes[0, i].set_ylabel('Price', fontsize=12)
        corr = city_data['Carpet Area (Sq-ft)'].corr(city_data['Price'])
        axes[0, i].annotate(f'Correlation: {corr:.2f}', xy=(0.05, 0.95), xycoords='axes fraction')
        
        # Log scale plot
        sns.scatterplot(x='Carpet Area (Sq-ft)', y='Price', data=city_data, alpha=0.6, ax=axes[1, i])
        axes[1, i].set_xscale('log')
        axes[1, i].set_yscale('log')
        axes[1, i].set_title(f'{city}: Log Carpet Area vs Log Price', fontsize=14)
        axes[1, i].set_xlabel('Log Carpet Area (Sq-ft)', fontsize=12)
        axes[1, i].set_ylabel('Log Price', fontsize=12)
        log_corr = np.log(city_data['Carpet Area (Sq-ft)']).corr(np.log(city_data['Price']))
        axes[1, i].annotate(f'Log Correlation: {log_corr:.2f}', xy=(0.05, 0.95), xycoords='axes fraction')
    
    plt.tight_layout()
    plt.show()
    plt.close()

def create_avg_price_by_property_type_chart(data):
    """Create a bar chart showing average price per sq ft by property type for Thane and Mumbai."""
    avg_price_by_type = data.groupby(['City', 'Type of Property'])['Price per sq ft'].mean().unstack()
    
    plt.figure(figsize=(12, 6))
    avg_price_by_type.plot(kind='bar')
    plt.title('Average Price per sq ft by Property Type', fontsize=16)
    plt.xlabel('City', fontsize=12)
    plt.ylabel('Average Price per sq ft', fontsize=12)
    plt.legend(title='Property Type', bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()
    plt.close()

analyze_thane_mumbai_property_data(data)

## Question 4

In [ ]:
def remove_outliers_iqr(df, columns):
    """
    Remove outliers from specified columns using the Interquartile Range (IQR) method.

    Args:
    df (pandas.DataFrame): The dataframe to process.
    columns (list): List of column names to remove outliers from.

    Returns:
    pandas.DataFrame: The dataframe with outliers removed.
    """
    for column in columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        df = df[(df[column] >= Q1 - 1.5 * IQR) & (df[column] <= Q3 + 1.5 * IQR)]
    return df

def analyze_property_data(data):
    """
    Main function to analyze and visualize property data across multiple cities.

    Args:
    data (pandas.DataFrame): The raw property data.
    """
    columns_to_clean = ['Price', 'Carpet Area', 'isPrimeLocationProperty', 'City', 'Area Name']
    cleaned_data = clean_and_prepare_data(data, columns_to_clean)

    create_distribution_analysis(cleaned_data)
    create_top_localities_chart(cleaned_data, 'isPrimeLocationProperty', 'Y', 'Top 5 Localities with Most Prime Properties by City')
    create_top_localities_chart(cleaned_data, 'isPrimeLocationProperty', 'N', 'Top 5 Localities with Most Non-Prime Properties by City')
    create_extreme_localities_chart(cleaned_data, 'Price_per_carpet_area', 5, True, 'Top 5 Most Expensive Localities by Price per Carpet Area for Each City')
    create_extreme_localities_chart(cleaned_data, 'Price_per_carpet_area', 5, False, 'Bottom 5 Least Expensive Localities by Price per Carpet Area for Each City')

def clean_and_prepare_data(data, columns_to_clean):
    """
    Clean and prepare the raw property data for analysis.

    Args:
    data (pandas.DataFrame): The raw property data.
    columns_to_clean (list): List of column names to clean.

    Returns:
    pandas.DataFrame: The cleaned and prepared data.
    """
    cleaned_data = data.dropna(subset=columns_to_clean, how='all')
    cleaned_data['Price'] = pd.to_numeric(cleaned_data['Price'], errors='coerce')
    cleaned_data['Carpet Area'] = pd.to_numeric(cleaned_data['Carpet Area'], errors='coerce')
    cleaned_data['Price'] = cleaned_data['Price'].fillna(cleaned_data['Price'].median())
    cleaned_data['Carpet Area'] = cleaned_data['Carpet Area'].fillna(cleaned_data['Carpet Area'].median())
    
    for col in ['isPrimeLocationProperty', 'City', 'Area Name']:
        cleaned_data[col] = cleaned_data[col].fillna(cleaned_data[col].mode()[0])
    
    cleaned_data['Price_per_carpet_area'] = cleaned_data['Price'] / cleaned_data['Carpet Area']
    
    columns_to_remove_outliers = ['Price', 'Carpet Area', 'Price_per_carpet_area']
    cleaned_data = remove_outliers_iqr(cleaned_data, columns_to_remove_outliers)
    
    return cleaned_data

def create_distribution_analysis(data):
    """Create box plots for distribution of price, carpet area, and price per carpet area."""
    fig, axes = plt.subplots(3, 1, figsize=(15, 20))
    metrics = ['Price', 'Carpet Area', 'Price_per_carpet_area']

    for i, metric in enumerate(metrics):
        sns.boxplot(x='City', y=metric, hue='isPrimeLocationProperty', data=data, ax=axes[i])
        axes[i].set_title(f'Distribution of {metric} by City and Prime Status')
        axes[i].set_xlabel('City')
        axes[i].set_ylabel(metric)
        axes[i].legend(title='Prime Location')
        axes[i].tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.show()
    plt.close()

def create_top_localities_chart(data, column, value, title):
    """Create a bar chart of top localities based on a specific criterion."""
    top_localities = data.groupby(['City', 'Area Name'])[column].apply(lambda x: (x == value).sum()).reset_index()
    top_localities = top_localities.groupby('City').apply(lambda x: x.nlargest(5, column)).reset_index(drop=True)
    top_localities = top_localities.sort_values(['City', column])

    plot_localities_chart(top_localities, column, title, f'Number of {"Prime" if value == "Y" else "Non-Prime"} Properties')

def create_extreme_localities_chart(data, column, n, ascending, title):
    """Create a bar chart of extreme (top or bottom) localities based on a specific metric."""
    extreme_localities = data.groupby(['City', 'Area Name'])[column].mean().reset_index()
    extreme_localities = extreme_localities.groupby('City').apply(lambda x: x.nlargest(n, column) if ascending else x.nsmallest(n, column)).reset_index(drop=True)
    extreme_localities = extreme_localities.sort_values(['City', column], ascending=[True, ascending])

    plot_localities_chart(extreme_localities, column, title, 'Price per Carpet Area')

def plot_localities_chart(data, column, title, ylabel):
    """Helper function to plot locality charts."""
    plt.figure(figsize=(20, 10))

    cities = data['City'].unique()
    colors = plt.cm.rainbow(np.linspace(0, 1, len(cities)))
    city_color = dict(zip(cities, colors))

    x_pos = 0
    x_ticks = []
    x_labels = []

    for city in cities:
        city_data = data[data['City'] == city]
        for _, row in city_data.iterrows():
            plt.bar(x_pos, row[column], color=city_color[city])
            x_ticks.append(x_pos)
            x_labels.append(row['Area Name'])
            x_pos += 1
        x_pos += 1

    plt.title(title)
    plt.xlabel('Locality Names')
    plt.ylabel(ylabel)
    plt.xticks(x_ticks, x_labels, rotation=90, ha='right')

    legend_elements = [plt.Rectangle((0,0),1,1, color=color, label=city) for city, color in city_color.items()]
    plt.legend(handles=legend_elements, title='Cities', bbox_to_anchor=(1.05, 1), loc='upper left')

    plt.tight_layout()
    plt.show()
    plt.close()

analyze_property_data(data)

## Question 5

In [ ]:
def convert_area_to_sqft(area, unit):
    """
    Convert the given area to square feet based on the unit.

    Args:
    area (float): The area value to be converted.
    unit (str): The unit of the area ('Sq-ft', 'Sq-m', or 'Sq-yrd').

    Returns:
    float or None: The area in square feet, or None if input is invalid.
    """
    if pd.isna(area) or pd.isna(unit):
        return None
    conversion_factors = {"Sq-ft": 1, "Sq-m": 10.7639, "Sq-yrd": 9}
    return area * conversion_factors.get(unit, 0)

def prepare_data_for_analysis(data):
    """
    Prepare the property data for analysis by adding derived columns.

    Args:
    data (pandas.DataFrame): The raw property data.

    Returns:
    pandas.DataFrame: The prepared data with additional columns.
    """
    df = data.copy()
    df["Carpet Area (Sq-ft)"] = df.apply(
        lambda row: convert_area_to_sqft(row["Carpet Area"], row["Carpet Area Unit"]), axis=1
    )
    df = df.dropna(subset=["Carpet Area (Sq-ft)"])
    
    df["Log Carpet Area"] = np.log10(df["Carpet Area"])
    df["Log Price"] = np.log10(df["Price"])
    df["Area_per_Unit_Price"] = df["Carpet Area"] / df["Price"]
    df["Log Area_per_Unit_Price"] = np.log10(df["Area_per_Unit_Price"])
    
    return df

def plot_log_price_vs_carpet_area(data):
    """
    Create a scatter plot of log-scaled price vs carpet area by city.

    Args:
    data (pandas.DataFrame): The prepared property data.
    """
    plt.figure(figsize=(12, 8))
    sns.scatterplot(
        data=data,
        x="Log Carpet Area",
        y="Log Price",
        hue="City",
        palette="Set1",
        s=100,
    )
    plt.title("Log-Scaled Price vs Carpet Area by City", fontsize=16)
    plt.xlabel("Log Carpet Area (log10 scale)", fontsize=12)
    plt.ylabel("Log Price (log10 scale)", fontsize=12)
    plt.grid(True)
    plt.show()

def plot_average_area_per_unit_price(data):
    """
    Create a bar plot of average carpet area per unit price by city.

    Args:
    data (pandas.DataFrame): The prepared property data.
    """
    plt.figure(figsize=(12, 8))
    sns.barplot(x="City", y="Area_per_Unit_Price", data=data, palette="coolwarm")
    plt.title("Average Carpet Area per Unit Price by City", fontsize=16)
    plt.xlabel("City", fontsize=12)
    plt.ylabel("Carpet Area per Unit Price (sq. ft. per currency unit)", fontsize=12)
    plt.xticks(rotation=45)
    plt.show()

def plot_log_area_per_unit_price_distribution(data):
    """
    Create a box plot of log-scaled distribution of carpet area per unit price by city.

    Args:
    data (pandas.DataFrame): The prepared property data.
    """
    plt.figure(figsize=(12, 8))
    sns.boxplot(x="City", y="Log Area_per_Unit_Price", data=data, palette="Set2")
    plt.title("Log-Scaled Distribution of Carpet Area per Unit Price by City", fontsize=16)
    plt.xlabel("City", fontsize=12)
    plt.ylabel("Log Carpet Area per Unit Price (log10 scale)", fontsize=12)
    plt.xticks(rotation=45)
    plt.show()

def analyze_and_visualize_property_data(data):
    """
    Main function to analyze and visualize property data.

    Args:
    data (pandas.DataFrame): The raw property data.
    """
    prepared_data = prepare_data_for_analysis(data)
    plot_log_price_vs_carpet_area(prepared_data)
    plot_average_area_per_unit_price(prepared_data)
    plot_log_area_per_unit_price_distribution(prepared_data)

analyze_and_visualize_property_data(data)

## Question 6

In [ ]:
def prepare_commercial_data(data, amenities_columns):
    """
    Prepare commercial property data for analysis.

    Args:
    data (pd.DataFrame): The input dataframe containing property data.
    amenities_columns (list): List of amenity column names.

    Returns:
    pd.DataFrame: Prepared commercial property data.
    """
    commercial_data = data[data['Commercial'] == 'Y'].copy()

    for col in amenities_columns:
        commercial_data[col] = commercial_data[col].astype(str).map({'1': 1, '1.0': 1, 'Y': 1, 'y': 1, 'yes': 1, 'Yes': 1}).fillna(0).astype(int)

    commercial_data['Amenity_Score'] = commercial_data[amenities_columns].sum(axis=1)

    features = ['Price', 'sqft Price ', 'Covered Area', 'Carpet Area', 'Floor No', 'Maintenance Charges',
                'Booking Amount', 'floors', 'Bathroom', 'balconies', 'Amenity_Score']

    for feature in features:
        commercial_data[feature] = pd.to_numeric(commercial_data[feature], errors='coerce')

    return commercial_data, features

def analyze_city_correlations(city_data, city_name, features):
    """
    Analyze correlations between features for a specific city.

    Args:
    city_data (pd.DataFrame): Data for a specific city.
    city_name (str): Name of the city.
    features (list): List of feature names to analyze.

    Returns:
    pd.DataFrame: Correlation matrix for the city.
    """
    correlation_matrix = city_data[features].corr()

    plt.figure(figsize=(12, 10))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
    plt.title(f'Correlation Heatmap of Commercial Properties in {city_name}')
    plt.tight_layout()
    plt.show()

    print(f"\nCorrelation Matrix for {city_name}:")
    print(correlation_matrix)

    strong_correlations = (correlation_matrix.abs() > 0.5) & (correlation_matrix != 1.0)
    strong_pairs = np.where(strong_correlations)
    print(f"\nStrong correlations in {city_name}:")
    for i, j in zip(*strong_pairs):
        if i < j:
            print(f"{features[i]} and {features[j]}: {correlation_matrix.iloc[i, j]:.2f}")

    print(f"\nKey statistics for {city_name}:")
    for feature in features:
        mean_val = city_data[feature].mean()
        median_val = city_data[feature].median()
        skew_val = city_data[feature].skew()
        print(f"{feature}: Mean = {mean_val:.2f}, Median = {median_val:.2f}, Skewness = {skew_val:.2f}")

    return correlation_matrix

def compare_city_correlations(mumbai_corr, thane_corr, features):
    """
    Compare correlations between Mumbai and Thane.

    Args:
    mumbai_corr (pd.DataFrame): Correlation matrix for Mumbai.
    thane_corr (pd.DataFrame): Correlation matrix for Thane.
    features (list): List of feature names.
    """
    correlation_diff = mumbai_corr - thane_corr
    plt.figure(figsize=(12, 10))
    sns.heatmap(correlation_diff, annot=True, cmap='coolwarm', vmin=-1, vmax=1, center=0)
    plt.title('Difference in Correlations (Mumbai - Thane)')
    plt.tight_layout()
    plt.show()

    print("\nMajor differences in correlations between Mumbai and Thane:")
    for i in range(len(features)):
        for j in range(i+1, len(features)):
            diff = correlation_diff.iloc[i, j]
            if abs(diff) > 0.2:  
                print(f"{features[i]} and {features[j]}: Difference of {diff:.2f}")

def plot_feature_distributions(mumbai_data, thane_data, features):
    """
    Plot feature distributions for Mumbai and Thane.

    Args:
    mumbai_data (pd.DataFrame): Data for Mumbai.
    thane_data (pd.DataFrame): Data for Thane.
    features (list): List of feature names to plot.
    """
    for feature in features:
        plt.figure(figsize=(10, 6))
        sns.histplot(mumbai_data[feature], kde=True, label='Mumbai')
        sns.histplot(thane_data[feature], kde=True, label='Thane')
        plt.title(f'Distribution of {feature} in Mumbai and Thane')
        plt.legend()
        plt.show()

        t_stat, p_value = stats.ttest_ind(mumbai_data[feature].dropna(), thane_data[feature].dropna())
        print(f"\nt-test for {feature}:")
        print(f"t-statistic: {t_stat:.4f}")
        print(f"p-value: {p_value:.4f}")

def analyze_top_amenities(commercial_data, amenities_columns):
    """
    Analyze and plot top amenities in commercial properties with different colors for each bar.

    Args:
    commercial_data (pd.DataFrame): Commercial property data.
    amenities_columns (list): List of amenity column names.
    """
    top_amenities = commercial_data[amenities_columns].mean().sort_values(ascending=False).head(10)
    
    plt.figure(figsize=(14, 7))
    colors = sns.color_palette("husl", 10)
    bars = plt.bar(range(len(top_amenities)), top_amenities.values, color=colors)
    
    plt.title('Top 10 Most Common Amenities in Commercial Properties', fontsize=16)
    plt.ylabel('Proportion of Properties', fontsize=12)
    plt.xlabel('Amenities', fontsize=12)
    plt.xticks(range(len(top_amenities)), top_amenities.index, rotation=45, ha='right', fontsize=10)
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                 f'{height:.2f}',
                 ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()

def analyze_commercial_property_density(data, cities, top_n=5):
    """
    Analyze and visualize the density of commercial properties in top N localities of specified cities.

    Args:
    data (pd.DataFrame): The input dataframe containing property data.
    cities (list): List of cities to analyze.
    top_n (int): Number of top localities to display for each city.
    """
    city_data = data[data['City'].isin(cities)]
    locality_stats = calculate_locality_statistics(city_data)
    top_localities = get_top_localities(locality_stats, 'Commercial_Density', top_n)
    
    plot_property_statistics(top_localities, cities, top_n, 'Density', 'Commercial_Density', 
                             'Commercial Property Density (Commercial / Total Buildings)')
    
    print("\nTop Localities by Commercial Property Density:")
    print(top_localities[['City', 'Area Name', 'Commercial_Count', 'Total_Count', 'Commercial_Density']])

def analyze_commercial_property_count(data, cities, top_n=5):
    """
    Analyze and visualize the count of commercial properties in top N localities of specified cities.

    Args:
    data (pd.DataFrame): The input dataframe containing property data.
    cities (list): List of cities to analyze.
    top_n (int): Number of top localities to display for each city.
    """
    city_data = data[data['City'].isin(cities) & (data['Commercial'] == 'Y')]
    locality_counts = city_data.groupby(['City', 'Area Name']).size().reset_index(name='Commercial_Count')
    top_localities = get_top_localities(locality_counts, 'Commercial_Count', top_n)
    
    plot_property_statistics(top_localities, cities, top_n, 'Count', 'Commercial_Count', 
                             'Number of Commercial Properties')
    
    print("\nTop Localities by Commercial Property Count:")
    print(top_localities)

def calculate_locality_statistics(data):
    """Calculate commercial property statistics for each locality."""
    locality_stats = data.groupby(['City', 'Area Name']).agg({
        'Commercial': ['count', lambda x: (x == 'Y').sum()]
    }).reset_index()
    locality_stats.columns = ['City', 'Area Name', 'Total_Count', 'Commercial_Count']
    locality_stats['Commercial_Density'] = locality_stats['Commercial_Count'] / locality_stats['Total_Count']
    return locality_stats

def get_top_localities(data, sort_column, top_n):
    """Get the top N localities for each city based on the specified sorting column."""
    return data.groupby('City').apply(lambda x: x.nlargest(top_n, sort_column)).reset_index(drop=True)

def plot_property_statistics(data, cities, top_n, stat_type, y_column, y_label):
    """Create a bar plot for property statistics."""
    plt.figure(figsize=(15, 8))
    colors = ['#6A0572', '#AB83A1']
    x = range(len(data))
    plt.bar(x, data[y_column], color=[colors[cities.index(city)] for city in data['City']])
    
    plt.xticks(x, [f"{row['Area Name']}\n({row['City']})" for _, row in data.iterrows()], rotation=45, ha='right')
    plt.title(f'Top {top_n} Localities with Highest Commercial Property {stat_type} in {" and ".join(cities)}')
    plt.xlabel('Locality (City)')
    plt.ylabel(y_label)
    
    handles = [plt.Rectangle((0,0),1,1, color=colors[i], label=city) for i, city in enumerate(cities)]
    plt.legend(handles=handles, title='Cities')
    
    for i, v in enumerate(data[y_column]):
        plt.text(i, v, f'{v:.2f}' if stat_type == 'Density' else str(v), ha='center', va='bottom')
    
    plt.tight_layout()
    plt.show()

def comprehensive_commercial_analysis(data, amenities_columns, cities=['Mumbai', 'Thane'], top_n=5):
    """
    Perform comprehensive analysis of commercial properties.

    Args:
    data (pd.DataFrame): The input dataframe containing property data.
    amenities_columns (list): List of amenity column names.
    cities (list): List of cities to analyze (default: ['Mumbai', 'Thane']).
    top_n (int): Number of top localities to display for each city (default: 5).
    """
    print(f"Performing comprehensive analysis of commercial properties in {', '.join(cities)}...")

    commercial_data, features = prepare_commercial_data(data, amenities_columns)
    
    mumbai_data = commercial_data[commercial_data['City'] == 'Mumbai']
    thane_data = commercial_data[commercial_data['City'] == 'Thane']

    mumbai_corr = analyze_city_correlations(mumbai_data, 'Mumbai', features)
    thane_corr = analyze_city_correlations(thane_data, 'Thane', features)

    compare_city_correlations(mumbai_corr, thane_corr, features)
    plot_feature_distributions(mumbai_data, thane_data, features)
    analyze_top_amenities(commercial_data, amenities_columns)

    analyze_commercial_property_density(data, cities, top_n)
    analyze_commercial_property_count(data, cities, top_n)

amenities_columns = ['Power Back Up', 'Lift', 'Rain Water Harvesting', 'Club House', 'Swimming Pool', 'Gymnasium', 'Park', 'Parking', 'Security', 'Water Storage', 'Private Terrace/Garden', 'Vaastu Compliant', 'Service/Goods Lift', 'Air Conditioned', 'Visitor Parking', 'Intercom Facility', 'Maintenance Staff', 'Waste Disposal', 'Laundry Service', 'Internet/Wi-Fi Connectivity', 'DTH Television Facility', 'RO Water System', 'Banquet Hall', 'Bar/Lounge', 'Cafeteria/Food Court', 'Conference Room', 'Piped Gas', 'Jogging and Strolling Track', 'Outdoor Tennis Courts', 'Mansion', 'Downtown', 'Skyline View', 'Wrap Around Balcony', 'Island Kitchen Layout', 'Full Glass Wall', 'House help accommodation', 'Concierge Services', 'Garden View', 'Marble flooring', 'Well Furnished', 'Modular Kitchen', 'Helipad', 'Private pool ', 'Private Jaccuzi', 'Mini Cinema Theatre', 'Golf Course', 'Infinity Swimming Pool', 'Pool with temperature control ', 'Sea facing', 'Skydeck', 'Earth quake resistant', 'Theme based Architectures', 'Health club with Steam / Jaccuzi', 'Large Clubhouse ', 'Large Open space', 'Fingerprint Access', 'Grand Entrance lobby', 'Private Garage', 'Fireplace', 'Wine Cellar', 'Sky Villa', 'Water Front', 'Hilltop', 'Smart Home', 'Barbeque space', 'Kids play area', 'Library', 'Puja Room', 'Study', 'Servant Room', 'Store', 'Aerobics Room', 'Canopy Walk', 'Coffee Lounge & Restaurants', 'Dance Studio', 'Event Space & Amphitheatre', 'Flower Gardens', 'Guest Accommodation', 'Indoor Squash & Badminton Courts', 'Kids Play Area', 'Kids Play Pool With Water Slides', 'Library And Business Centre', 'Meditation Area', 'Multipurpose Courts', 'Recreational Pool', 'Rentable CommuniPfty Space', 'Retail Boulevard (Retail Shops)', 'Cycling & Jogging Track', 'Fire Fighting Equipment', 'Aesthetically designed landscape garden']
comprehensive_commercial_analysis(data, amenities_columns)

## Question 7

In [ ]:
def prepare_data(data, cities, amenities):
    """
    Prepare the data for analysis by filtering cities and converting price to numeric.
    
    Args:
    data (pd.DataFrame): The original dataset.
    cities (list): List of cities to analyze.
    amenities (list): List of amenities to analyze.
    
    Returns:
    pd.DataFrame: Prepared dataset for analysis.
    """
    city_data = data[data['City'].isin(cities)].copy()
    city_data['Price'] = pd.to_numeric(city_data['Price'], errors='coerce')
    return city_data

def plot_amenity_price_correlation(data, cities, amenities):
    """
    Plot the correlation between amenities and price for each city,
    using a different color for each amenity.
    
    Args:
    data (pd.DataFrame): Prepared dataset.
    cities (list): List of cities to analyze.
    amenities (list): List of amenities to analyze.
    """
    plt.figure(figsize=(16, 6 * len(cities)))
    
    color_palette = sns.color_palette("husl", len(amenities))
    
    for i, city in enumerate(cities):
        city_corr = data[data['City'] == city][amenities + ['Price']].corr()['Price'][:-1]
        ax = plt.subplot(len(cities), 1, i+1)
        bars = ax.bar(range(len(city_corr)), city_corr.values, color=color_palette)
    
        ax.set_title(f'Correlation between Amenities and Price in {city}', fontsize=16)
        ax.set_xlabel('Amenities', fontsize=12)
        ax.set_ylabel('Correlation with Price', fontsize=12)
        ax.set_xticks(range(len(city_corr)))
        ax.set_xticklabels(city_corr.index, rotation=45, ha='right', fontsize=10)
        
        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                    f'{height:.2f}',
                    ha='center', va='bottom', fontsize=9)
        
        ax.axhline(y=0, color='black', linestyle='--', linewidth=0.8, alpha=0.5)
        ax.grid(axis='y', linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.show()

def plot_average_price_comparison(data, cities, amenities):
    """
    Plot the average price comparison for properties with and without each amenity,
    using different colors for each amenity and improved label positioning.
    
    Args:
    data (pd.DataFrame): Prepared dataset.
    cities (list): List of cities to analyze.
    amenities (list): List of amenities to analyze.
    """
    plt.figure(figsize=(18, 6 * len(cities)))
    color_palette = sns.color_palette("husl", len(amenities))
    
    for i, city in enumerate(cities):
        city_df = data[data['City'] == city]
        ax = plt.subplot(len(cities), 1, i+1)
        
        bar_width = 0.35
        index = range(len(amenities))
        
        with_amenity_prices = []
        without_amenity_prices = []
        
        for amenity in amenities:
            with_amenity = city_df[city_df[amenity] == 1]['Price'].mean()
            without_amenity = city_df[city_df[amenity] == 0]['Price'].mean()
            with_amenity_prices.append(with_amenity)
            without_amenity_prices.append(without_amenity)
        
        with_bars = ax.bar(index, with_amenity_prices, bar_width, label='With Amenity', color=color_palette)
        without_bars = ax.bar([i + bar_width for i in index], without_amenity_prices, bar_width, 
                              label='Without Amenity', color=[plt.cm.get_cmap('Pastel1')(i) for i in range(len(amenities))])
        
        ax.set_title(f'Average Price Comparison in {city}', fontsize=16)
        ax.set_ylabel('Average Price (INR)', fontsize=12)
        ax.set_xlabel('Amenities', fontsize=12)
        ax.set_xticks([i + bar_width/2 for i in index])
        ax.set_xticklabels(amenities, rotation=45, ha='right', fontsize=10)
        
        def add_value_labels(bars):
            for bar in bars:
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., height,
                        f'{height:,.0f}',
                        ha='center', va='bottom', rotation=90, fontsize=8)
        
        add_value_labels(with_bars)
        add_value_labels(without_bars)
        
        ax.legend(fontsize=10)
        ax.set_ylim(0, max(max(with_amenity_prices), max(without_amenity_prices)) * 1.2)
    
    plt.tight_layout()
    plt.show()

def plot_amenity_frequency_comparison(data, cities, amenities):
    """
    Plot the frequency of each amenity across cities.
    
    Args:
    data (pd.DataFrame): Prepared dataset.
    cities (list): List of cities to analyze.
    amenities (list): List of amenities to analyze.
    """
    amenity_freq = data.groupby('City')[amenities].mean()
    plt.figure(figsize=(14, 8))
    amenity_freq.T.plot(kind='bar')
    plt.title('Amenity Frequency Comparison: Mumbai vs Thane', fontsize=16)
    plt.xlabel('Amenities', fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.legend(title='City', fontsize=10)
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.show()

def plot_price_distribution_for_top_amenities(data, cities, amenities, top_n=3):
    """
    Plot the normalized price distribution for properties with top amenities.
    
    Args:
    data (pd.DataFrame): Prepared dataset.
    cities (list): List of cities to analyze.
    amenities (list): List of amenities to analyze.
    top_n (int): Number of top amenities to consider.
    """
    amenity_freq = data.groupby('City')[amenities].mean()
    top_amenities = amenity_freq.mean().nlargest(top_n).index
    
    plt.figure(figsize=(16, 12))
    for i, city in enumerate(cities):
        city_df = data[data['City'] == city]
        plt.subplot(len(cities), 1, i+1)
        
        max_density = 0
        for amenity in top_amenities:
            prices = city_df[city_df[amenity] == 1]['Price']
            prices = prices[prices > 0]
            
            kde = sns.kdeplot(data=prices, log_scale=True, label=amenity)
            line = kde.lines[-1]
            x, y = line.get_data()
            max_density = max(max_density, np.max(y))
            y = y / max_density
            plt.plot(x, y, label=amenity)
        
        plt.title(f'Normalized Price Distribution for Properties with Top Amenities in {city}', fontsize=16)
        plt.xlabel('Price (INR) - Log Scale', fontsize=12)
        plt.ylabel('Normalized Density', fontsize=12)
        plt.legend(fontsize=10)
        plt.xscale('log')
        plt.xticks([1e6, 1e7, 1e8, 1e9], ['1M', '10M', '100M', '1B'])
        plt.ylim(0, 2)
    plt.tight_layout()
    plt.show()

def analyze_amenities_and_prices(data, cities=['Mumbai', 'Thane'], 
                                 amenities=['Swimming Pool', 'Gymnasium', 'Club House', 
                                            'Power Back Up', 'Lift', 'Park', 'Security']):
    """
    Perform comprehensive analysis of amenities and their relation to property prices.
    
    Args:
    data (pd.DataFrame): The original dataset.
    cities (list): List of cities to analyze.
    amenities (list): List of amenities to analyze.
    """
    prepared_data = prepare_data(data, cities, amenities)
    plot_amenity_price_correlation(prepared_data, cities, amenities)
    plot_average_price_comparison(prepared_data, cities, amenities)
    plot_amenity_frequency_comparison(prepared_data, cities, amenities)
    plot_price_distribution_for_top_amenities(prepared_data, cities, amenities)

amenities = ['Swimming Pool', 'Gymnasium', 'Club House', 'Power Back Up', 'Lift', 'Park', 'Security']
analyze_amenities_and_prices(data, cities=['Mumbai', 'Thane'], amenities=amenities)

## Part 8

In [ ]:
def prepare_data(data, cities=['Mumbai', 'Thane']):
    """
    Prepare the data for analysis by filtering cities and converting price to numeric.
    
    Args:
    data (pd.DataFrame): The original dataset.
    cities (list): List of cities to analyze.
    
    Returns:
    pd.DataFrame: Prepared dataset for analysis.
    """
    prepared_data = data[data['City'].isin(cities)].copy()
    prepared_data['Price'] = pd.to_numeric(prepared_data['Price'], errors='coerce')
    prepared_data = remove_outliers_iqr(prepared_data, ['Price'])
    prepared_data['Possession Category'] = prepared_data['Possession Status'].apply(categorize_possession)
    prepared_data['Availability Category'] = prepared_data['Availability Starts From'].apply(categorize_availability)
    return prepared_data

def remove_outliers_iqr(df, columns):
    """
    Remove outliers from specified columns using the Interquartile Range (IQR) method.
    
    Args:
    df (pd.DataFrame): Input dataframe.
    columns (list): List of column names to remove outliers from.
    
    Returns:
    pd.DataFrame: Dataframe with outliers removed.
    """
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

def categorize_possession(status):
    """
    Categorize possession status.
    
    Args:
    status (str): Original possession status.
    
    Returns:
    str: Categorized possession status.
    """
    if pd.isna(status):
        return 'Unknown'
    elif status in ['Ready to Move', 'Immediately']:
        return 'Ready to Move'
    else:
        return 'Under Construction'

def categorize_availability(date):
    """
    Categorize availability based on the date.
    
    Args:
    date (str): Original availability date.
    
    Returns:
    str: Categorized availability.
    """
    if pd.isna(date):
        return 'Unknown'
    try:
        date = datetime.strptime(date, "%b '%y")
        now = datetime.now()
        if date <= now:
            return 'Already Available'
        elif date <= now + pd.DateOffset(years=1):
            return 'Available within 1 year'
        else:
            return 'Available after 1 year'
    except:
        return 'Unknown'

def analyze_prices(df, group_cols):
    """
    Analyze prices by grouping data and calculating statistics.
    
    Args:
    df (pd.DataFrame): Input dataframe.
    group_cols (list): Columns to group by.
    
    Returns:
    pd.DataFrame: Analysis results.
    """
    result = df.groupby(group_cols)['Price'].agg(['mean', 'median', 'count']).reset_index()
    result['mean'] = result['mean'].apply(lambda x: f"₹{x:,.0f}")
    result['median'] = result['median'].apply(lambda x: f"₹{x:,.0f}")
    return result

def plot_price_distribution(data, hue_col, title):
    """
    Plot price distribution using a box plot.
    
    Args:
    data (pd.DataFrame): Input dataframe.
    hue_col (str): Column name for hue in the plot.
    title (str): Title of the plot.
    """
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='City', y='Price', hue=hue_col, data=data)
    plt.title(title)
    plt.ylabel('Price (₹)')
    plt.yscale('log')
    plt.show()
    plt.close()

def perform_ttest(df, city, category_col):
    """
    Perform t-test for price differences between categories in a city.
    
    Args:
    df (pd.DataFrame): Input dataframe.
    city (str): City to analyze.
    category_col (str): Column name for categories.
    
    Returns:
    str: T-test result.
    """
    city_data = df[df['City'] == city]
    categories = city_data[category_col].unique()
    if len(categories) < 2:
        return f"Not enough categories for t-test in {city}"
    group1 = city_data[city_data[category_col] == categories[0]]['Price']
    group2 = city_data[city_data[category_col] == categories[1]]['Price']
    t_stat, p_value = stats.ttest_ind(group1, group2)
    return f"T-test for {city} ({categories[0]} vs {categories[1]}): p-value = {p_value:.4f}"

def calculate_price_difference(data, city):
    """
    Calculate price difference percentage between Ready-to-Move and Under-Construction properties.
    
    Args:
    data (pd.DataFrame): Input dataframe.
    city (str): City to analyze.
    
    Returns:
    float: Price difference percentage.
    """
    city_data = data[data['City'] == city]
    ready_price = city_data[city_data['Possession Category'] == 'Ready to Move']['Price'].mean()
    under_construction_price = city_data[city_data['Possession Category'] == 'Under Construction']['Price'].mean()
    return ((ready_price - under_construction_price) / under_construction_price) * 100

def analyze_property_data(data):
    """
    Analyze property data, including price analysis, visualizations, and statistical tests.
    
    Args:
    data (pd.DataFrame): Original dataset.
    """
    prepared_data = prepare_data(data)
    
    possession_analysis = analyze_prices(prepared_data, ['City', 'Possession Category'])
    availability_analysis = analyze_prices(prepared_data, ['City', 'Availability Category'])
    
    print("Price Analysis by Possession Category and City (after removing outliers):")
    print(possession_analysis.to_string(index=False))
    print("\nPrice Analysis by Availability Category and City (after removing outliers):")
    print(availability_analysis.to_string(index=False))
    
    plot_price_distribution(prepared_data, 'Possession Category', 'Property Prices: Ready-to-Move vs Under-Construction in Mumbai and Thane')
    plot_price_distribution(prepared_data, 'Availability Category', 'Property Prices by Availability Category in Mumbai and Thane')
    
    print("\nStatistical Tests:")
    print(perform_ttest(prepared_data, 'Mumbai', 'Possession Category'))
    print(perform_ttest(prepared_data, 'Thane', 'Possession Category'))
    
    for city in ['Mumbai', 'Thane']:
        price_diff_percent = calculate_price_difference(prepared_data, city)
        print(f"\n{city} - Price difference percentage (Ready-to-Move vs Under-Construction): {price_diff_percent:.2f}%")
    
    print("\nNumber of properties in each category after removing outliers:")
    print(prepared_data.groupby(['City', 'Possession Category']).size().unstack(fill_value=0))
    print("\nNumber of properties in each availability category after removing outliers:")
    print(prepared_data.groupby(['City', 'Availability Category']).size().unstack(fill_value=0))

analyze_property_data(data)

## Part 9

In [ ]:
def remove_outliers_iqr(df, columns):
    """
    Remove outliers from specified columns using the Interquartile Range (IQR) method.
    
    Args:
    df (pd.DataFrame): Input dataframe.
    columns (list): List of column names to remove outliers from.
    
    Returns:
    pd.DataFrame: Dataframe with outliers removed.
    """
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    return df

def prepare_data(data):
    """
    Prepare the data for analysis by converting price to numeric, creating amenity columns,
    and removing outliers.
    
    Args:
    data (pd.DataFrame): The original dataset.
    
    Returns:
    pd.DataFrame: Prepared dataset for analysis.
    list: List of amenity column names.
    """
    prepared_data = data.copy()
    prepared_data['Price'] = pd.to_numeric(prepared_data['Price'], errors='coerce')
    prepared_data = remove_outliers_iqr(prepared_data, ['Price'])
    
    amenity_columns = [
        "Power Back Up", "Lift", "Rain Water Harvesting", "Club House",
        "Swimming Pool", "Gymnasium", "Park", "Parking", "Security", "Water Storage",
    ]
    
    for column in amenity_columns:
        prepared_data[column] = prepared_data[column].notnull().astype(int)
    
    prepared_data["Amenity_Count"] = prepared_data[amenity_columns].sum(axis=1)
    prepared_data["Log_Price"] = np.log10(prepared_data["Price"])
    
    return prepared_data, amenity_columns

def get_top_developers(data, n=10):
    """
    Get the top n developers by average property price.
    
    Args:
    data (pd.DataFrame): Prepared dataset.
    n (int): Number of top developers to return.
    
    Returns:
    pd.Series: Top n developers by average property price.
    """
    data_filtered = data[data["Developer"].notna() & (data["Developer"] != "NA")]
    return (
        data_filtered.groupby("Developer")["Price"]
        .mean()
        .sort_values(ascending=False)
        .head(n)
    )

def plot_top_developers_avg_price(top_developers):
    """
    Plot average property price for top developers with different bar colors.
    
    Args:
    top_developers (pd.Series): Top developers by average property price.
    """
    plt.figure(figsize=(12, 6))
    colors = sns.color_palette("husl", len(top_developers))
    bars = plt.bar(top_developers.index, top_developers.values, color=colors)
    
    plt.title("Top 10 Developers by Average Property Price", fontsize=16)
    plt.xlabel("Developer", fontsize=12)
    plt.ylabel("Average Price (INR)", fontsize=12)
    plt.xticks(rotation=45, ha="right")
    
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                 f'₹{height:,.0f}',
                 ha='center', va='bottom', rotation=0, fontsize=10)
    
    plt.grid(axis='y', linestyle='--', alpha=0.7)    
    plt.tight_layout()
    plt.show()
    plt.close()

def plot_price_distribution_by_developer(data, top_developers):
    """
    Plot price distribution for top developers.
    
    Args:
    data (pd.DataFrame): Prepared dataset.
    top_developers (pd.Series): Top developers by average property price.
    """
    plt.figure(figsize=(14, 8))
    sns.boxplot(
        x="Developer",
        y="Log_Price",
        data=data[data["Developer"].isin(top_developers.index)],
    )
    plt.title("Distribution of Property Prices by Top Developers (Log Scale)")
    plt.xlabel("Developer")
    plt.ylabel("Log10(Price) in INR")
    plt.xticks(rotation=45, ha="right")
    yticks = plt.gca().get_yticks()
    yticklabels = [f"{10**y:,.0f}" for y in yticks]
    plt.gca().set_yticklabels(yticklabels)
    plt.grid(axis="y", linestyle="--", alpha=0.7)
    plt.tight_layout()
    plt.show()
    plt.close()

def plot_amenities_heatmap(data, top_developers, amenity_columns):
    """
    Plot heatmap of amenities offered by top developers.
    
    Args:
    data (pd.DataFrame): Prepared dataset.
    top_developers (pd.Series): Top developers by average property price.
    amenity_columns (list): List of amenity column names.
    """
    amenities_by_developer = (
        data[data["Developer"].isin(top_developers.index)]
        .groupby("Developer")[amenity_columns]
        .mean()
    )
    plt.figure(figsize=(12, 8))
    sns.heatmap(amenities_by_developer, annot=True, cmap="YlGnBu", fmt=".2f")
    plt.title("Amenities Offered by Top Developers")
    plt.ylabel("Developer")
    plt.xlabel("Amenity")
    plt.tight_layout()
    plt.show()
    plt.close()

def plot_price_vs_amenities(data, top_developers):
    """
    Plot scatter plot of price vs number of amenities by developer.
    
    Args:
    data (pd.DataFrame): Prepared dataset.
    top_developers (pd.Series): Top developers by average property price.
    """
    plt.figure(figsize=(14, 8))
    sns.scatterplot(
        x="Amenity_Count",
        y="Log_Price",
        hue="Developer",
        data=data[data["Developer"].isin(top_developers.index)],
        alpha=0.7,
    )
    plt.title("Price vs Number of Amenities by Developer (Log Scale)")
    plt.xlabel("Number of Amenities")
    plt.ylabel("Log10(Price) in INR")
    yticks = plt.gca().get_yticks()
    yticklabels = [f"{10**y:,.0f}" for y in yticks]
    plt.gca().set_yticklabels(yticklabels)
    plt.grid(linestyle="--", alpha=0.7)
    plt.legend(title="Developer", bbox_to_anchor=(1.05, 1), loc="upper left")
    plt.tight_layout()
    plt.show()
    plt.close()

def plot_developer_distribution(data, top_n=11):
    """
    Plot pie chart of property distribution among top developers.
    
    Args:
    data (pd.DataFrame): Prepared dataset.
    top_n (int): Number of top developers to show individually.
    """
    developer_counts = data["Developer"].value_counts()
    other_count = developer_counts.iloc[top_n:].sum()
    top_developers_counts = developer_counts.iloc[:top_n]
    top_developers_counts["Other"] = other_count
    total_properties = top_developers_counts.sum()
    percentages = top_developers_counts / total_properties * 100
    
    plt.figure(figsize=(12, 8))
    plt.pie(
        percentages,
        labels=percentages.index,
        autopct="%1.1f%%",
        startangle=90,
        pctdistance=0.85,
    )
    center_circle = plt.Circle((0, 0), 0.70, fc="white")
    fig = plt.gcf()
    fig.gca().add_artist(center_circle)
    plt.title(f"Distribution of Number of Properties by Top {top_n} Developers")
    plt.axis("equal")
    plt.legend(title="Developers", loc="center left", bbox_to_anchor=(1, 0, 0.5, 1))
    plt.tight_layout()
    plt.show()
    plt.close()

def analyze_developers_and_amenities(data):
    """
    Perform comprehensive analysis of developers and amenities.
    
    Args:
    data (pd.DataFrame): The original dataset.
    """
    prepared_data, amenity_columns = prepare_data(data)
    top_developers = get_top_developers(prepared_data)
    
    plot_top_developers_avg_price(top_developers)
    plot_price_distribution_by_developer(prepared_data, top_developers)
    plot_amenities_heatmap(prepared_data, top_developers, amenity_columns)
    plot_price_vs_amenities(prepared_data, top_developers)
    plot_developer_distribution(prepared_data)

analyze_developers_and_amenities(data)